# [ VAE ] OOD Detection using FSS

In [13]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.getcwd() + '/core')
sys.path.append(os.getcwd() + '/core/train_GLOW') 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import core.config as config
from core.model_loader import load_pretrained_VAE
from core.data_loader import TRAIN_loader, TEST_loader
from core.custom_loss import KL_div, VAE_loss_pixel
from core.fisher_utils_VAE import Calculate_fisher_VAE, Calculate_score_VAE, AUTO_VAE
from core.visualize import plot_hist, AUROC, plot_scores_all_layers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'torch'

# Define Global Variables & Initialize

In [8]:
# FISHER_invs : Fisher inverse matrices from calculating Fisher score (w.r.t. train-dist)
FISHERs = {'cifar10': {}, 'fmnist': {}}
         
# NORMalize_FACtors : Normalizing Factor (w.r.t. train-dist)
NORM_FACs = {'cifar10': {}, 'fmnist': {}}
             
# SCOREs : (Scalars) Scores from calculating Fisher score (w.r.t. target-dist)
SCOREs = {'cifar10': {}, 'fmnist': {}}


# VAE-CIFAR10

In [9]:
opt = config.VAE_cifar10
netE, netG = load_pretrained_VAE(option=opt.train_dist, ngf=64, nz=100, beta=1)
netE.eval()
netG.eval()

params = {
    #'Emain0_w': netE.main[0].weight,
    #'Emain1_w': netE.main[1].weight,
    #'Emain1_b': netE.main[1].bias,
    #'Emain3_w': netE.main[3].weight,
    #'Emain4_w': netE.main[4].weight,
    #'Emain4_b': netE.main[4].bias,
    #'Emain6_w': netE.main[6].weight,
    #'Emain7_w': netE.main[7].weight,
    #'Emain7_b': netE.main[7].bias,
    'Econv1_w': netE.conv1.weight,
    #'Econv1_b': netE.conv1.bias,
    #'Econv2_w': netE.conv2.weight,
    #'Econv2_b': netE.conv2.bias,
    #'Gmain0_w': netG.main[0].weight,
    #'Gmain1_w': netG.main[1].weight,
    #'Gmain1_b': netG.main[1].bias,
    #'Gmain3_w': netG.main[3].weight,
    #'Gmain4_w': netG.main[4].weight,
    #'Gmain4_b': netG.main[4].bias,
    #'Gmain6_w': netG.main[6].weight,
    #'Gmain7_w': netG.main[7].weight,
    #'Gmain7_b': netG.main[7].bias,
    #'Gmain9_w': netG.main[9].weight,
}


In [10]:
a, b, _scores = AUTO_VAE(opt, netE, netG, params, max_iter=[1000, 1000], loss_type='ELBO_pixel', method='Vanilla')

Files already downloaded and verified


Calculate Fisher VAE:   2%|█                                                    | 999/50000 [00:09<07:59, 102.19step/s]


Files already downloaded and verified


Calculate Score VAE:  10%|█████▍                                                | 999/10000 [00:09<01:28, 101.79step/s]


In [11]:
for pname in params.keys():
    FISHERs[opt.train_dist][pname] = a[pname] 
    NORM_FACs[opt.train_dist][pname] = b[pname]
for ood in opt.ood_list:
    SCOREs[opt.train_dist][ood] = _scores[ood]

In [12]:
# AUROC curve
auroc = {}
for pname in params.keys():
    _auroc = {}
    for ood in opt.ood_list:
        args = [
            SCOREs[opt.train_dist][opt.train_dist][pname],
            SCOREs[opt.train_dist][ood][pname],
        ]
        labels = [opt.train_dist, ood]
        _auroc[ood] = AUROC(*args, labels=labels, verbose=False)
    auroc[pname] = _auroc
    
auroc

{'Econv1_w': {'cifar10': 0.49999999999999994, 'lsun': 0.473313}}

In [6]:
# Just show scores
# plot_scores_all_layers(train_dist, params, SCOREs, opt, save=True)


# VAE-FMNIST

In [4]:
opt = config.VAE_fmnist
netE, netG = load_pretrained_VAE(option=opt.train_dist, ngf=32, nz=100, beta=1)
netE.eval()
netG.eval()

params = {
    #'Emain0_w': netE.main[0].weight,
    #'Emain1_w': netE.main[1].weight,
    #'Emain1_b': netE.main[1].bias,
    #'Emain3_w': netE.main[3].weight,
    #'Emain4_w': netE.main[4].weight,
    #'Emain4_b': netE.main[4].bias,
    #'Emain6_w': netE.main[6].weight,
    #'Emain7_w': netE.main[7].weight,
    #'Emain7_b': netE.main[7].bias,
    'Econv1_w': netE.conv1.weight,
    #'Econv1_b': netE.conv1.bias,
    #'Econv2_w': netE.conv2.weight,
    #'Econv2_b': netE.conv2.bias,
    #'Gmain0_w': netG.main[0].weight,
    #'Gmain1_w': netG.main[1].weight,
    #'Gmain1_b': netG.main[1].bias,
    #'Gmain3_w': netG.main[3].weight,
    #'Gmain4_w': netG.main[4].weight,
    #'Gmain4_b': netG.main[4].bias,
    #'Gmain6_w': netG.main[6].weight,
    #'Gmain7_w': netG.main[7].weight,
    #'Gmain7_b': netG.main[7].bias,
    #'Gmain9_w': netG.main[9].weight,
}

In [7]:
a, b, _scores = AUTO_VAE(opt, netE, netG, params, max_iter=[10000, 5000], loss_type='ELBO_pixel', method='SMW')

Calculate Score VAE:  50%|██████████████████████████▍                          | 4999/10000 [00:33<00:33, 148.39step/s]


Using downloaded and verified file: ../data\test_32x32.mat


Calculate Score VAE: 100%|██████████████████████████████████████████████████████| 3000/3000 [00:27<00:00, 108.14step/s]


Files already downloaded and verified


Calculate Score VAE:  50%|██████████████████████████▍                          | 4999/10000 [00:34<00:34, 143.65step/s]


Files already downloaded and verified


Calculate Score VAE:  50%|██████████████████████████▍                          | 4999/10000 [00:33<00:33, 148.55step/s]


Files already downloaded and verified


Calculate Score VAE:  50%|██████████████████████████▍                          | 4999/10000 [00:32<00:32, 154.88step/s]


In [9]:
for pname in params.keys():
    FISHERs[opt.train_dist][pname] = a[pname] 
    NORM_FACs[opt.train_dist][pname] = b[pname]
for ood in opt.ood_list:
    SCOREs[opt.train_dist][ood] = _scores[ood]

In [11]:
# AUROC curve
auroc = {}
for pname in params.keys():
    _auroc = {}
    for ood in opt.ood_list:
        args = [
            SCOREs[opt.train_dist][opt.train_dist][pname],
            SCOREs[opt.train_dist][ood][pname],
        ]
        labels = [opt.train_dist, ood]
        _auroc[ood] = AUROC(*args, labels=labels, verbose=False)
    auroc[pname] = _auroc
    
auroc

{'Econv1_w': {'fmnist': 0.5,
  'svhn': 0.9998754000000001,
  'celeba': 0.99931916,
  'lsun': 0.9989489333333333,
  'cifar10': 0.99863984,
  'cifar100': 0.9984999600000001,
  'mnist': 0.99330204,
  'kmnist': 0.99609384,
  'omniglot': 1.0,
  'notmnist': 0.99976164,
  'noise': 0.9811936,
  'constant': 0.99809412}}

In [ ]:
# Just show scores
# plot_scores_all_layers(train_dist, params, SCOREs, opt, save=True)